# Seismic event detection

## Problem statement

In this tutorial, we will analyse timeseries data recorded by an acoustic sensor in a lab friction experiment (courtesy of Evangelos Korkolis, Utrecht University). During this experiment, the sample exhibited stick-slip deformation, the laboratory-scale analogue of earthquakes. Just prior to a major stick-slip event, the sample would start to emit tiny acoustic emissions ("foreshocks"), which are detected by piezo-electric sensors ("seismometers") installed around the sample. Some of these acoustic emissions are clearly visible in the timeseries, whereas others occur near the level of the noise. Classical methods (such as ST/LT thresholding) don't perform well on the smaller acoustic emissions, and so we will use machine learning to lower the automatic detection threshold.

## Data description

The training/testing data are small, randomly selected segments of acoustic timeseries data, each segment consisting of 1024 datapoints. Associated with each segment is a binary label that is `0` when the segment only contains noise, and is `1` when the segment contains one or more acoustic emissions. The training and testing data are shuffled, so that there is no temporal relation between the segments.

In [ ]:
# Import the libraries
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import gzip
import pickle


def load_data():
    with gzip.GzipFile("AE_data.tar.gz", "r") as f:
        data = pickle.load(f)
    return data["train_data"], data["train_labels"], data["test_data"], data["test_labels"]


# Load the dataset
train_signals, train_labels, test_signals, test_labels = load_data()

# For convolutional layers, an additional dimension needs to be added
train_signals = np.expand_dims(train_signals, 2)
test_signals = np.expand_dims(test_signals, 2)

print(train_signals.shape, test_signals.shape)

## Data visualisation

As always, it's good to first have a look at some examples from the training data set. The code below plots "event" signals on the left, and "noise" signals on the right:

In [ ]:
# Select training signals of events
events = (train_labels == 1)
event_signals = train_signals[events][:5].reshape((5, -1))

# Select training signals of noise (no events)
noise_signals = train_signals[~events][:5].reshape((5, -1))

# Plot 5 examples of events and noise
for i in range(5):
    # Events
    plt.subplot(5, 2, 1+2*i)
    if i == 0:
        plt.title("Events")
    plt.plot(event_signals[i])
    plt.xticks([])
    plt.yticks([])
    
    # Noise
    plt.subplot(5, 2, 2+2*i)
    if i == 0:
        plt.title("Noise")
    plt.plot(noise_signals[i])
    plt.xticks([])
    plt.yticks([])

plt.tight_layout()
plt.show()

## Model construction

Your task will be to construct a neural network (dense, convolutional, or a combination of both) that takes a 1D timeseries of size 1024 as an input, and outputs a single scalar between 0 and 1 (0 corresponds to "no event", 1 corresponds to "yes event"). In the code below, the output layer is given, everything else is up to you.

In [ ]:
# Keras default initialiser = glorot_uniform
# A better initialiser for ReLU activations = he_normal
initializer = "he_normal"

model = keras.Sequential([
    # Insert your architecture here
    # Examples of layers you could include:
    #
    # Fully-connected:  keras.layers.Dense(128, activation=tf.nn.relu, kernel_initializer=initializer)
    # Convolutional:    keras.layers.Conv1D(64, kernel_size=5, activation=tf.nn.relu, kernel_initializer=initializer, padding="same")
    # Max pooling:      keras.layers.MaxPooling1D()
    # Flattening:       keras.layers.Flatten()
    #
    
    # Output layer: 1 output, sigmoid activation
    keras.layers.Dense(1, activation=tf.nn.sigmoid, kernel_initializer="glorot_normal")
])

# Compile and print a summary
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
print(model.summary())

## Training

Make sure to experiment with different training durations (`epochs`) to ensure that the model was trained long enough, but not too long...

In [ ]:
model.fit(
    train_signals, 
    train_labels, 
    validation_data=(test_signals, test_labels),
    verbose=1,
    epochs=10)

## Test accuracy and visualisation

The worst-case performance of the model would be a 50-50 guess (yes/no event), which would give a test accuracy of 50% (since events and noise are evenly represented in the test set). So our trained model should perform significantly better than 50% to be useful.

In [ ]:
test_loss, test_acc = model.evaluate(test_signals, test_labels)
print("Test accuracy: %.4f" % test_acc)

Just to see what this accuracy score looks like, we can plot a number of seismograms and colour them according to the accuracy of the prediction (green = correct, red = wrong).

In [ ]:
predictions = model.predict(test_signals)
predictions = np.round(predictions).astype(int)

fig = plt.figure(figsize=(14, 12))

# Plot 70 examples of events and noise
for i in range(10):
    for j in range(7):
        n = 7*i + j
        # Events
        plt.subplot(10, 7, 1+n)
        if predictions[n] == test_labels[n]:
            colour = "g"
        else:
            colour = "r"
        plt.plot(test_signals[n], c=colour)
        plt.xticks([])
        plt.yticks([])

plt.tight_layout()
plt.show()